In [1]:
import pandas as pd
import numpy as np
import scipy as sp


## actionType 和 actionTimeDiff
    > goodfeature = ['last1Type','last2Type','last3Type','last4Type','last5Type',
               'last6Type','last7Type','last8Type','last9Type','last10Type',
               'diffTimeAvg','diffTimeStd','diffTimeVar','diffTimeSkew',
               'diffTimeKurt','diffTimeMin','diffTimeMax','last1DiffTime','last2DiffTime', 
               'last3DiffTime', 'last4DiffTime', 'last5DiffTime','last6DiffTime',
               'last7DiffTime','last8DiffTime','last9DiffTime','last10DiffTime',
               'last11DiffTime','last12DiffTime','last13DiffTime','last14DiffTime',
               'last15DiffTime','diffTimeMaxSubMin','diffTimeMaxSubAvg','diffTimeAvgSubMin',
               ]




In [2]:
df_train_data = pd.read_csv('../data/train/orderFuture_train.csv')
df_test_data = pd.read_csv('../data/test/orderFuture_test.csv')


In [3]:
# 首先选择action信息
df_train_action = pd.read_csv('../data/train/action_train.csv')
df_test_action = pd.read_csv('../data/test/action_test.csv')


In [4]:
# actionType对应的特征，[倒数后5个Type, 前2个actionType]
column = ['userid','last1Type','last2Type','last3Type','last4Type','last5Type',
          'last6Type','last7Type','last8Type','last9Type','last10Type',]

df_train_actionType = pd.DataFrame(columns=column)

userlist = list(set(df_train_action['userid']))
k = 0
for user in userlist:
    k+=1
    if k%1000 == 0:
        print(k)
    
    df_users = df_train_action[df_train_action['userid'] == user]
    df_users = df_users.sort_values('actionTime')
    
    row = [user]
    tmp_type = []
    tmp_type.extend(df_users['actionType'].values[-10:])
    
    # 注意要时间进行发转，统计的时候是到这统计的。
    tmp_type.reverse()
    
    length = len(tmp_type)
    if len(tmp_type) < 10:
        tmp_type.extend([tmp_type[length-1] for i in range(10-len(tmp_type))])
                
    row.extend(tmp_type)    
    
    df_train_actionType = df_train_actionType.append(pd.DataFrame([row],columns=column),ignore_index=True)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [5]:
# actionType对应的特征，[倒数后5个Type, 前2个actionType]
column = ['userid','last1Type','last2Type','last3Type','last4Type','last5Type',
          'last6Type','last7Type','last8Type','last9Type','last10Type',]

df_test_actionType = pd.DataFrame(columns=column)

userlist = list(set(df_test_action['userid']))
k = 0
for user in userlist:
    k+=1
    if k%1000 == 0:
        print(k)
    
    df_users = df_test_action[df_test_action['userid'] == user]
    df_users = df_users.sort_values('actionTime')
    
    row = [user]
    tmp_type = []
    tmp_type.extend(df_users['actionType'].values[-10:])
    
    # 注意要时间进行发转，统计的时候是到这统计的。
    tmp_type.reverse()

    length = len(tmp_type)
    if len(tmp_type) < 10:
        tmp_type.extend([tmp_type[length-1] for i in range(10-len(tmp_type))])
            
    row.extend(tmp_type)    
    
    df_test_actionType = df_test_actionType.append(pd.DataFrame([row],columns=column),ignore_index=True)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [6]:
df_train_actionType.info()
df_test_actionType.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40307 entries, 0 to 40306
Data columns (total 11 columns):
userid        40307 non-null object
last1Type     40307 non-null object
last2Type     40307 non-null object
last3Type     40307 non-null object
last4Type     40307 non-null object
last5Type     40307 non-null object
last6Type     40307 non-null object
last7Type     40307 non-null object
last8Type     40307 non-null object
last9Type     40307 non-null object
last10Type    40307 non-null object
dtypes: object(11)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10076 entries, 0 to 10075
Data columns (total 11 columns):
userid        10076 non-null object
last1Type     10076 non-null object
last2Type     10076 non-null object
last3Type     10076 non-null object
last4Type     10076 non-null object
last5Type     10076 non-null object
last6Type     10076 non-null object
last7Type     10076 non-null object
last8Type     10076 non-null object
last9Type     10076 n

In [7]:

df_train_data = pd.merge(df_train_data,df_train_actionType, how='left',on='userid')
df_test_data = pd.merge(df_test_data,df_test_actionType, how='left',on='userid')


In [8]:
###--------------- actionTime差值的特征 --------------###

In [9]:
def diffInfo(x):
    if len(x) == 0:
        return x
    
    # 所有时间的最大值
    cur = 1505087865
    
    x = list(x)
    x.append(cur)
#     print(x)
    for i in range(len(x)-1):
        x[i] = x[i+1] - x[i]
    
    return np.array(x[:-1])


In [10]:
# 时间间隔特征，[avg,std,var,skew,kurt,min,max, last1, last2,last3,last4,last5]

column = ['userid','diffTimeAvg','diffTimeStd','diffTimeVar','diffTimeSkew','diffTimeKurt',
          'diffTimeMin','diffTimeMax','last1DiffTime','last2DiffTime','last3DiffTime',
          'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
          'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime','last13DiffTime',
          'last14DiffTime','last15DiffTime']

df_train_diffTime = pd.DataFrame(columns=column)

userlist = list(set(df_train_action['userid']))

k = 0
for user in userlist:
    k+=1
    if k%1000 == 0:
        print(k)

    df_users_t = df_train_action[df_train_action['userid'] == user]
    df_users_t = df_users_t.sort_values('actionTime')
    
    df_users_t['actionTimeDiff'] = diffInfo(df_users_t['actionTime'])
    
    row = [user]
    
    row.append(df_users_t['actionTimeDiff'].mean())
    row.append(df_users_t['actionTimeDiff'].std())
    row.append(df_users_t['actionTimeDiff'].var())
    row.append(df_users_t['actionTimeDiff'].skew())
    row.append(df_users_t['actionTimeDiff'].kurt())
    row.append(df_users_t['actionTimeDiff'].min())
    row.append(df_users_t['actionTimeDiff'].max())
    
    # 注意要时间进行发转，统计的时候是到这统计的。
    tmp_time = []
    tmp_time.extend(df_users_t['actionTimeDiff'].values[-15:])
    tmp_time.reverse()

    # 用差值的平均数填充
    if len(tmp_time) < 15:
        tmp_time.extend([row[1] for i in range(15-len(tmp_time))])
    
    row.extend(tmp_time)
    
    df_train_diffTime = df_train_diffTime.append(pd.DataFrame([row],columns=column),ignore_index=True)
    


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [11]:
# 极差(max-min),(max-avg),(avg-min)
df_train_diffTime['diffTimeMaxSubMin'] = df_train_diffTime['diffTimeMax'] - df_train_diffTime['diffTimeMin']
df_train_diffTime['diffTimeMaxSubAvg'] = df_train_diffTime['diffTimeMax'] - df_train_diffTime['diffTimeAvg']
df_train_diffTime['diffTimeAvgSubMin'] = df_train_diffTime['diffTimeAvg'] - df_train_diffTime['diffTimeMin']




In [12]:
# 时间间隔特征，[avg,std,var,skew,kurt,min,max, last1, last2,last3,last4,last5]

column = ['userid','diffTimeAvg','diffTimeStd','diffTimeVar','diffTimeSkew','diffTimeKurt',
          'diffTimeMin','diffTimeMax','last1DiffTime','last2DiffTime','last3DiffTime',
          'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
          'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime','last13DiffTime',
          'last14DiffTime','last15DiffTime']

df_test_diffTime = pd.DataFrame(columns=column)

userlist = list(set(df_test_action['userid']))

k = 0
for user in userlist:
    k+=1
    if k%1000 == 0:
        print(k)

    df_users_t = df_test_action[df_test_action['userid'] == user]
    df_users_t = df_users_t.sort_values('actionTime')
    
    df_users_t['actionTimeDiff'] = diffInfo(df_users_t['actionTime'])
    
    row = [user]
    
    row.append(df_users_t['actionTimeDiff'].mean())
    row.append(df_users_t['actionTimeDiff'].std())
    row.append(df_users_t['actionTimeDiff'].var())
    row.append(df_users_t['actionTimeDiff'].skew())
    row.append(df_users_t['actionTimeDiff'].kurt())
    row.append(df_users_t['actionTimeDiff'].min())
    row.append(df_users_t['actionTimeDiff'].max())
    
    
    # 注意要时间进行发转，统计的时候是到这统计的。
    tmp_time = []
    tmp_time.extend(df_users_t['actionTimeDiff'].values[-15:])
    tmp_time.reverse()

    # 用差值的平均数填充
    if len(tmp_time) < 15:
        tmp_time.extend([row[1] for i in range(15-len(tmp_time))])
    
    row.extend(tmp_time)
    
    df_test_diffTime = df_test_diffTime.append(pd.DataFrame([row],columns=column),ignore_index=True)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [13]:
# 极差(max-min),(max-avg),(avg-min)
df_test_diffTime['diffTimeMaxSubMin'] = df_test_diffTime['diffTimeMax'] - df_test_diffTime['diffTimeMin']
df_test_diffTime['diffTimeMaxSubAvg'] = df_test_diffTime['diffTimeMax'] - df_test_diffTime['diffTimeAvg']
df_test_diffTime['diffTimeAvgSubMin'] = df_test_diffTime['diffTimeAvg'] - df_test_diffTime['diffTimeMin']


In [14]:
df_train_diffTime.info()
df_test_diffTime.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40307 entries, 0 to 40306
Data columns (total 26 columns):
userid               40307 non-null object
diffTimeAvg          40307 non-null float64
diffTimeStd          39047 non-null float64
diffTimeVar          39047 non-null float64
diffTimeSkew         38288 non-null float64
diffTimeKurt         37196 non-null float64
diffTimeMin          40307 non-null object
diffTimeMax          40307 non-null object
last1DiffTime        40307 non-null object
last2DiffTime        40307 non-null object
last3DiffTime        40307 non-null object
last4DiffTime        40307 non-null object
last5DiffTime        40307 non-null object
last6DiffTime        40307 non-null object
last7DiffTime        40307 non-null object
last8DiffTime        40307 non-null object
last9DiffTime        40307 non-null object
last10DiffTime       40307 non-null object
last11DiffTime       40307 non-null object
last12DiffTime       40307 non-null object
last13DiffTime       40307

In [15]:
df_train_diffTime.fillna(0,inplace=True)
df_test_diffTime.fillna(0,inplace=True)

In [16]:
df_train_data = pd.merge(df_train_data,df_train_diffTime, how='left',on='userid')

df_test_data = pd.merge(df_test_data,df_test_diffTime, how='left',on='userid')


In [17]:
df_train_diffTime.columns



Index(['userid', 'diffTimeAvg', 'diffTimeStd', 'diffTimeVar', 'diffTimeSkew',
       'diffTimeKurt', 'diffTimeMin', 'diffTimeMax', 'last1DiffTime',
       'last2DiffTime', 'last3DiffTime', 'last4DiffTime', 'last5DiffTime',
       'last6DiffTime', 'last7DiffTime', 'last8DiffTime', 'last9DiffTime',
       'last10DiffTime', 'last11DiffTime', 'last12DiffTime', 'last13DiffTime',
       'last14DiffTime', 'last15DiffTime', 'diffTimeMaxSubMin',
       'diffTimeMaxSubAvg', 'diffTimeAvgSubMin'],
      dtype='object')

In [18]:
df_train_data.info()
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 37 columns):
userid               40307 non-null object
orderType            40307 non-null int64
last1Type            40307 non-null object
last2Type            40307 non-null object
last3Type            40307 non-null object
last4Type            40307 non-null object
last5Type            40307 non-null object
last6Type            40307 non-null object
last7Type            40307 non-null object
last8Type            40307 non-null object
last9Type            40307 non-null object
last10Type           40307 non-null object
diffTimeAvg          40307 non-null float64
diffTimeStd          40307 non-null float64
diffTimeVar          40307 non-null float64
diffTimeSkew         40307 non-null float64
diffTimeKurt         40307 non-null float64
diffTimeMin          40307 non-null int64
diffTimeMax          40307 non-null int64
last1DiffTime        40307 non-null int64
last2DiffTime        40307 non

In [21]:
df_train_data.drop('orderType',axis=1,inplace=True)
df_train_data.to_csv('../data/dataSet/df_train_1.csv',index=False,index_label=False)
df_test_data.to_csv('../data/dataSet/df_test_1.csv',index=False,index_label=False)

